In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
import nltk
from nltk.tokenize import word_tokenize
from stop_words import get_stop_words
from nltk.stem import WordNetLemmatizer
from gensim import corpora, models
import gensim
import glob2
from itertools import chain
import os
import numpy as np
import itertools
import cPickle as pickle
from  scipy.stats import rankdata
import copy
from openpyxl import Workbook
from openpyxl.compat import range
from openpyxl.utils import get_column_letter
from openpyxl import load_workbook
import pandas as pd
from scipy.stats import hmean
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize


2018-03-20 19:34:44,519 : INFO : 'pattern' package not found; tag filters are not available for English


In [3]:
# Remove non-alphabets from text
def only_alphabet(text):
    return ''.join(i for i in text if (ord(i)<123 and ord(i)>96) or (ord(i)<91 and ord(i)>64) or ord(i)==32) 

In [4]:
full_filenames = []
x= ['LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt','LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt' ,'LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt']
for glob_filenames in x:
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        full_filenames.append(filename)
            
filenames = []
x= ['LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt','LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt' ,'LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt']
for glob_filenames in x:
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        filenames.append(os.path.basename(filename))

en_stop = set(get_stop_words('en'))
ps = PorterStemmer()

# "yield" for each file return token list  i.e list of lists
def files_to_tokens(glob_filenames):
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        f = open(filename)
        # read the whole file as lowercase string
        string = only_alphabet(f.read()).lower()
        tokens = []
        # tokenize that string
        for word in word_tokenize(string):
            if word not in en_stop:
                tokens.append(ps.stem(word))

        yield tokens
        f.close()


        
# yields token list for files specific to courts; needed for creating dictionaries
class texts:
    def DCDRC(self):
        return files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt')
    def NCDRC(self):
        return files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt')
    def SCDRC(self):
        return files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt')
    def DelhiHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt')
    def JharkhandHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt')
    def JodhpurHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt')
    def KolkataHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt')
    def SupremeCourt(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt')

# yields bow for each file - tuples id,fq ; needed to train models   
class my_corpus:    
    def DCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/*.txt'):
            yield dictionary.doc2bow(text)
    def NCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt'):
            yield dictionary.doc2bow(text)            
    def SCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def DelhiHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def JharkhandHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def JodhpurHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def KolkataHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def SupremeCourt(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt'):
            yield dictionary.doc2bow(text)
    def everything(self):
        return chain(self.DCDRC(), self.NCDRC(), self.SCDRC(), self.DelhiHC(),
                     self.JharkhandHC(), self.JodhpurHC(), self.KolkataHC(), self.SupremeCourt())
        
        

In [5]:
#################################### laod the dictinary
dictionary = corpora.Dictionary.load('stem100_dictionary')

2018-03-20 19:35:08,438 : INFO : loading Dictionary object from stem100_dictionary
2018-03-20 19:35:08,456 : INFO : loaded stem100_dictionary


In [6]:
######################################## load BOW
corpus = corpora.MmCorpus('stem100.mm')

2018-03-20 19:35:08,564 : INFO : loaded corpus index from stem100.mm.index
2018-03-20 19:35:08,565 : INFO : initializing corpus reader from stem100.mm
2018-03-20 19:35:08,566 : INFO : accepted corpus with 351985 documents, 40069 features, 92572684 non-zero entries


In [7]:
%%time
# Tfidf model
# Will take time as to generate a model it processes over the whole corpus
tfidf = gensim.models.TfidfModel(corpus)
# save tfidf model
tfidf.save('tfidf_stem100.mdl')

2018-03-20 19:35:08,595 : INFO : collecting document frequencies
2018-03-20 19:35:08,599 : INFO : PROGRESS: processing document #0
2018-03-20 19:35:23,054 : INFO : PROGRESS: processing document #10000
2018-03-20 19:35:39,874 : INFO : PROGRESS: processing document #20000
2018-03-20 19:35:58,249 : INFO : PROGRESS: processing document #30000
2018-03-20 19:36:15,841 : INFO : PROGRESS: processing document #40000
2018-03-20 19:36:28,301 : INFO : PROGRESS: processing document #50000
2018-03-20 19:36:44,184 : INFO : PROGRESS: processing document #60000
2018-03-20 19:36:57,491 : INFO : PROGRESS: processing document #70000
2018-03-20 19:37:10,328 : INFO : PROGRESS: processing document #80000
2018-03-20 19:37:26,657 : INFO : PROGRESS: processing document #90000
2018-03-20 19:37:46,180 : INFO : PROGRESS: processing document #100000
2018-03-20 19:38:03,260 : INFO : PROGRESS: processing document #110000
2018-03-20 19:38:14,086 : INFO : PROGRESS: processing document #120000
2018-03-20 19:38:28,980 : 

CPU times: user 7min 38s, sys: 1.47 s, total: 7min 40s
Wall time: 7min 57s


In [9]:
##################################### load tfidf model
tfidf = gensim.models.tfidfmodel.TfidfModel.load('tfidf_stem100.mdl')

2018-03-20 19:47:38,730 : INFO : loading TfidfModel object from tfidf_stem100.mdl
2018-03-20 19:47:39,042 : INFO : loaded tfidf_stem100.mdl


In [10]:
%%time
# tfidf wrapper coprus
# doesn't take time - yield
tfidf_corpus = tfidf[corpus]

# actual tfidf corpus serialized
corpora.MmCorpus.serialize('tfidf_stem100.mm', tfidf_corpus)


2018-03-20 19:47:44,962 : INFO : storing corpus in Matrix Market format to tfidf_stem100.mm
2018-03-20 19:47:45,027 : INFO : saving sparse matrix to tfidf_stem100.mm
2018-03-20 19:47:45,031 : INFO : PROGRESS: saving document #0
2018-03-20 19:47:48,659 : INFO : PROGRESS: saving document #1000
2018-03-20 19:47:51,493 : INFO : PROGRESS: saving document #2000
2018-03-20 19:47:55,018 : INFO : PROGRESS: saving document #3000
2018-03-20 19:48:00,060 : INFO : PROGRESS: saving document #4000
2018-03-20 19:48:04,038 : INFO : PROGRESS: saving document #5000
2018-03-20 19:48:06,646 : INFO : PROGRESS: saving document #6000
2018-03-20 19:48:10,954 : INFO : PROGRESS: saving document #7000
2018-03-20 19:48:16,588 : INFO : PROGRESS: saving document #8000
2018-03-20 19:48:21,076 : INFO : PROGRESS: saving document #9000
2018-03-20 19:48:26,059 : INFO : PROGRESS: saving document #10000
2018-03-20 19:48:30,942 : INFO : PROGRESS: saving document #11000
2018-03-20 19:48:34,997 : INFO : PROGRESS: saving docum

2018-03-20 19:56:17,119 : INFO : PROGRESS: saving document #122000
2018-03-20 19:56:22,518 : INFO : PROGRESS: saving document #123000
2018-03-20 19:56:28,729 : INFO : PROGRESS: saving document #124000
2018-03-20 19:56:35,634 : INFO : PROGRESS: saving document #125000
2018-03-20 19:56:41,314 : INFO : PROGRESS: saving document #126000
2018-03-20 19:56:45,845 : INFO : PROGRESS: saving document #127000
2018-03-20 19:56:49,737 : INFO : PROGRESS: saving document #128000
2018-03-20 19:56:54,154 : INFO : PROGRESS: saving document #129000
2018-03-20 19:56:57,372 : INFO : PROGRESS: saving document #130000
2018-03-20 19:56:59,995 : INFO : PROGRESS: saving document #131000
2018-03-20 19:57:02,512 : INFO : PROGRESS: saving document #132000
2018-03-20 19:57:05,369 : INFO : PROGRESS: saving document #133000
2018-03-20 19:57:08,146 : INFO : PROGRESS: saving document #134000
2018-03-20 19:57:10,822 : INFO : PROGRESS: saving document #135000
2018-03-20 19:57:13,311 : INFO : PROGRESS: saving document #13

2018-03-20 20:03:55,863 : INFO : PROGRESS: saving document #245000
2018-03-20 20:03:56,865 : INFO : PROGRESS: saving document #246000
2018-03-20 20:03:57,994 : INFO : PROGRESS: saving document #247000
2018-03-20 20:03:59,197 : INFO : PROGRESS: saving document #248000
2018-03-20 20:04:00,502 : INFO : PROGRESS: saving document #249000
2018-03-20 20:04:01,861 : INFO : PROGRESS: saving document #250000
2018-03-20 20:04:03,200 : INFO : PROGRESS: saving document #251000
2018-03-20 20:04:04,506 : INFO : PROGRESS: saving document #252000
2018-03-20 20:04:05,645 : INFO : PROGRESS: saving document #253000
2018-03-20 20:04:06,959 : INFO : PROGRESS: saving document #254000
2018-03-20 20:04:08,227 : INFO : PROGRESS: saving document #255000
2018-03-20 20:04:09,147 : INFO : PROGRESS: saving document #256000
2018-03-20 20:04:10,332 : INFO : PROGRESS: saving document #257000
2018-03-20 20:04:11,362 : INFO : PROGRESS: saving document #258000
2018-03-20 20:04:12,462 : INFO : PROGRESS: saving document #25

CPU times: user 21min 12s, sys: 6.6 s, total: 21min 18s
Wall time: 22min


In [11]:
##################################### load tfidf corpus
tfidf_corpus = corpora.MmCorpus('tfidf_stem100.mm')

2018-03-20 20:09:45,220 : INFO : loaded corpus index from tfidf_stem100.mm.index
2018-03-20 20:09:45,231 : INFO : initializing corpus reader from tfidf_stem100.mm
2018-03-20 20:09:45,234 : INFO : accepted corpus with 351985 documents, 40069 features, 92572684 non-zero entries


In [12]:
# get the query truth in dictionary
# query no. gives the list of 0-1 documents
query_truth = {}
for i in range(1,11):
    query_truth[str(i)]=[]
    
f = open('LegalAdhocTask/Consumer.qrels')
lines = [line.rstrip('\n').split("\t") for line in f]
for line in lines:
    del line[1]
    query_truth[line[0]].append(line[1:])


In [13]:
# create index - Tfidf
tfidf_index = gensim.similarities.Similarity('tfidf.index',tfidf_corpus,len(dictionary))

# save index - tfidf
gensim.similarities.Similarity.save(tfidf_index,'TFIDF_index')


2018-03-21 12:24:22,674 : INFO : starting similarity index under tfidf.index
2018-03-21 12:24:42,486 : INFO : PROGRESS: fresh_shard size=10000
2018-03-21 12:25:06,440 : INFO : PROGRESS: fresh_shard size=20000
2018-03-21 12:25:27,799 : INFO : PROGRESS: fresh_shard size=30000
2018-03-21 12:25:32,852 : INFO : creating sparse index
2018-03-21 12:25:32,853 : INFO : creating sparse matrix from corpus
2018-03-21 12:25:32,854 : INFO : PROGRESS: at document #0/32768
2018-03-21 12:25:38,927 : INFO : PROGRESS: at document #10000/32768
2018-03-21 12:25:45,248 : INFO : PROGRESS: at document #20000/32768
2018-03-21 12:25:51,685 : INFO : PROGRESS: at document #30000/32768
2018-03-21 12:25:53,144 : INFO : created <32768x40069 sparse matrix of type '<type 'numpy.float32'>'
	with 10181509 stored elements in Compressed Sparse Row format>
2018-03-21 12:25:53,160 : INFO : creating sparse shard #0
2018-03-21 12:25:53,161 : INFO : saving index shard to tfidf.index.0
2018-03-21 12:25:53,161 : INFO : saving Sp

2018-03-21 12:35:42,445 : INFO : PROGRESS: at document #0/32768
2018-03-21 12:35:49,714 : INFO : PROGRESS: at document #10000/32768
2018-03-21 12:35:57,827 : INFO : PROGRESS: at document #20000/32768
2018-03-21 12:36:01,117 : INFO : PROGRESS: at document #30000/32768
2018-03-21 12:36:01,337 : INFO : created <32768x40069 sparse matrix of type '<type 'numpy.float32'>'
	with 9540344 stored elements in Compressed Sparse Row format>
2018-03-21 12:36:01,338 : INFO : creating sparse shard #6
2018-03-21 12:36:01,339 : INFO : saving index shard to tfidf.index.6
2018-03-21 12:36:01,341 : INFO : saving SparseMatrixSimilarity object under tfidf.index.6, separately None
2018-03-21 12:36:02,032 : INFO : saved tfidf.index.6
2018-03-21 12:36:02,033 : INFO : loading SparseMatrixSimilarity object from tfidf.index.6
2018-03-21 12:36:02,226 : INFO : loaded tfidf.index.6
2018-03-21 12:36:02,284 : INFO : PROGRESS: fresh_shard size=0
2018-03-21 12:36:10,243 : INFO : PROGRESS: fresh_shard size=10000
2018-03-2

In [14]:
########################### load tfidf index
tfidf_index = gensim.similarities.Similarity.load('TFIDF_index')

2018-03-21 12:40:05,958 : INFO : loading Similarity object from TFIDF_index
2018-03-21 12:40:05,959 : INFO : loaded TFIDF_index


In [15]:
# calculate similarity for all 
tfidf_index.num_best= None

In [16]:
#######################         SIMILIRATIES SCORES AND RANKING    ###########################
tfidf_results = copy.deepcopy(query_truth)
### lists of list (indexing from 1, dummy element)
sim_list = [0]
rank_list = [0]
queries = [0]
for i,query in enumerate(files_to_tokens('LegalAdhocTask/q*.txt')):
    queries.append(query)
    # inside the square bracket determines query representation
    
    sims = tfidf_index[tfidf[dictionary.doc2bow(query)]]
    sim_list.append(sims)
    # rank of every document wrt similarity
    ranks = rankdata(sims, method='ordinal')
    ranks= len(ranks)+1 - ranks 
    rank_list.append(ranks)
    
    # update the query truth tuples with similarity score and the ranks
    for x in tfidf_results[str(i+1)]:
        x.append(sims[filenames.index(x[0]+'.txt')])
        x.append(ranks[filenames.index(x[0]+'.txt')])
        #x.append(common_words(filenames.index(x[0]+'.txt'), tfidf_corpus, query))
    
    # sort wrt relevance(from truth) and then ranks(from our model)
    #tfidf_results[str(i+1)].sort(key=lambda x: (-int(x[1]),x[3]))     
    
        

2018-03-21 12:40:07,424 : INFO : loading SparseMatrixSimilarity object from tfidf.index.0
2018-03-21 12:40:08,852 : INFO : loaded tfidf.index.0
2018-03-21 12:40:08,934 : INFO : loading SparseMatrixSimilarity object from tfidf.index.1
2018-03-21 12:40:09,713 : INFO : loaded tfidf.index.1
2018-03-21 12:40:09,756 : INFO : loading SparseMatrixSimilarity object from tfidf.index.2
2018-03-21 12:40:09,764 : INFO : loading index from tfidf.index.2.index.npy with mmap=r
2018-03-21 12:40:09,821 : INFO : loaded tfidf.index.2
2018-03-21 12:40:10,520 : INFO : loading SparseMatrixSimilarity object from tfidf.index.3
2018-03-21 12:40:11,296 : INFO : loaded tfidf.index.3
2018-03-21 12:40:11,340 : INFO : loading SparseMatrixSimilarity object from tfidf.index.4
2018-03-21 12:40:11,932 : INFO : loaded tfidf.index.4
2018-03-21 12:40:11,965 : INFO : loading SparseMatrixSimilarity object from tfidf.index.5
2018-03-21 12:40:11,978 : INFO : loading index from tfidf.index.5.index.npy with mmap=r
2018-03-21 12:

In [18]:
tfidf_results


{'1': [['ConsumerCourt_DCDRC_100385', '1', 0.20793898, 382],
  ['ConsumerCourt_DCDRC_106530', '0', 0.12333998, 1408],
  ['ConsumerCourt_DCDRC_107608', '1', 0.11513183, 1621],
  ['ConsumerCourt_DCDRC_114291', '1', 0.18758003, 565],
  ['ConsumerCourt_DCDRC_114382', '1', 0.13899776, 1145],
  ['ConsumerCourt_DCDRC_118185', '1', 0.05366122, 5978],
  ['ConsumerCourt_DCDRC_130318', '0', 0.17913997, 644],
  ['ConsumerCourt_DCDRC_130570', '1', 0.2935077, 55],
  ['ConsumerCourt_DCDRC_131146', '1', 0.10124508, 2026],
  ['ConsumerCourt_DCDRC_131717', '1', 0.12602413, 1348],
  ['ConsumerCourt_DCDRC_131741', '1', 0.11566975, 1607],
  ['ConsumerCourt_DCDRC_131818', '1', 0.14040397, 1120],
  ['ConsumerCourt_DCDRC_131950', '0', 0.080463998, 2945],
  ['ConsumerCourt_DCDRC_131972', '0', 0.11535629, 1615],
  ['ConsumerCourt_DCDRC_132932', '0', 0.10744458, 1834],
  ['ConsumerCourt_DCDRC_133592', '1', 0.23412392, 208],
  ['ConsumerCourt_DCDRC_134386', '0', 0.47163835, 1],
  ['ConsumerCourt_DCDRC_135474', '1

In [19]:
from openpyxl import Workbook
from openpyxl.compat import range
from openpyxl.utils import get_column_letter
from openpyxl import load_workbook

In [20]:
# Store results in 1st sheet
wb = Workbook()
dest_filename = 'tfidf.xlsx'
ws1 = wb.active
ws1.title = "TFIDF"
ws1.append(['Query','Filename', 'Relevance', 'Score', 'Rank'])
for key,value in tfidf_results.iteritems():
    for i in value:
        ws1.append([int(key)]+ i)

wb.save(filename = dest_filename)

In [21]:
#evaluation_Qwise - Precision, Recall and Fscore

import pandas as pd

f = pd.ExcelFile('tfidf.xlsx')

from scipy.stats import hmean



model_name = 'tfidf'
places = 4
wb = load_workbook('tfidf.xlsx')
ws1 = wb.create_sheet(title="evaluation_Qwise")

ws1.append(['Query','Docs considered', 'Precision/Recall', 'Model', 1, 5 ,10, 25, 50, 100, 500, 1000])

k = [1.0, 5.0 ,10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]
for sheet in f.sheet_names:
    if(sheet == "evaluation"):
        continue
    for q in np.arange(1,11):

        x = f.parse(sheet)
        x = x[x['Query'] == q]
        total_1 = (float)(x[x['Relevance'] == 1].shape[0])
        total_0 = (float)(x[x['Relevance'] == 0].shape[0])
        total = (float)(x.shape[0])
        
        ####  repitition due to complication in considering both 1 & 0 relevance
        # precision 1
        row = [q, 1, 'P', model_name + ' :' + sheet]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 1 , x['Rank'] <= i)].shape[0]/i)
        ws1.append(row);
        p1 = row;
        # precision 0
        row = [q, 0, 'P', model_name + ' :' + sheet]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 0 , x['Rank'] <= i)].shape[0]/i)    
        ws1.append(row);
        p0 = row;
        #precision 10
        row = [q, 10, 'P', model_name + ' :' + sheet]
        for i in k:
            row.append(x[x['Rank'] <= i].shape[0]/i)  
        ws1.append(row);
        p10 = row;  
        
        
        # recall 1
        row = [q, 1, 'R', model_name + ' :' + sheet]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 1 , x['Rank'] <= i)].shape[0]/total_1)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r1 = row;    
        # recall 0
        row = [q, 0, 'R', model_name + ' :' + sheet]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 0 , x['Rank'] <= i)].shape[0]/total_0)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r0 = row;
        #recall 10
        row = [q, 10, 'R', model_name + ' :' + sheet]
        for i in k:
            row.append(x[x['Rank'] <= i].shape[0]/total)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r10 = row;      


        # F 1
        row = [q, 1, 'F', model_name + ' :' + sheet]
        for i in range(-len(k), 0, 1):
            if p1[i] == 0.0 :
                row.append(0.0)
            else:
                row.append(hmean([p1[i],r1[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);

        # F 0
        row = [q, 0, 'F', model_name + ' :' + sheet]
        for i in range(-len(k), 0, 1):
            if p0[i] == 0.0:
                row.append(0.0)
            else:
                row.append(hmean([p0[i],r0[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);

        # F 10
        row = [q, 10, 'F', model_name + ' :' + sheet]
        for i in range(-len(k), 0, 1):
            if p10[i] == 0.0:
                row.append(0.0)
            else:
                row.append(hmean([p10[i],r10[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);
        
wb.save(filename = 'tfidf.xlsx')

In [22]:
#Query wise Average precision
# precision sum
def p_sum(z):
    z = z.copy()  
    z.sort_values(inplace=True)
    result = 0
    for i,val in enumerate(z):
        result += (i+1)/float(val) 
    return result


import pandas as pd

f = pd.ExcelFile('tfidf.xlsx')

from scipy.stats import hmean


model_name = 'tfidf'
places = 4
wb = load_workbook('tfidf.xlsx')
ws1 = wb.create_sheet(title="AP_Qwise")

ws1.append(['Query','Docs considered', 'Precision/Recall', 'Model', 1, 5 ,10, 25, 50, 100, 500, 1000])


for sheet in f.sheet_names:
    
    if(sheet == "evaluation"or sheet == "evaluation_Qwise"):
        continue
    for q in np.arange(1,11):

        x = f.parse(sheet)
        x = x[x['Query'] == q]
        total = {}
        total[1] = (float)(x[x['Relevance'] == 1].shape[0])
        total[0] = (float)(x[x['Relevance'] == 0].shape[0])
        total[10] = (float)(x.shape[0])
        
        for rel in [1,0]:
            # precision 1
            row = [q, rel, 'AP', model_name + ' :' + sheet]
            for i in [1.0, 5.0, 10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]:
                row.append(p_sum(x[np.logical_and(x['Relevance'] == rel, x['Rank'] <= i)]['Rank'])/total[rel])
            ws1.append(row);

        row = [q, 10, 'AP', model_name + ' :' + sheet]
        for i in [1.0, 5.0, 10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]:
            row.append(p_sum(x[x['Rank'] <= i]['Rank'])/total[10])
        ws1.append(row);
wb.save(filename = 'tfidf.xlsx')